<a href="https://colab.research.google.com/github/19K41A0503/AI/blob/main/503_Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ASSIGNMENT-5

Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing libraries

In [2]:
import numpy as np
import random
import pandas as pd
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
from sklearn import metrics

Load Data

In [3]:
data = pd.read_csv('/content/drive/MyDrive/data1-5.csv')
data.head()

,X1,X2,X3,Y
0,5551.82208,4983.17184,4888.39680,5072.95872
1,4983.17184,4888.39680,5072.95872,5196.25980
2,4888.39680,5072.95872,5196.25980,5641.29720
3,5072.95872,5196.25980,5641.29720,7153.17732
4,5196.25980,5641.29720,7153.17732,7849.80504



Outliers Detection

In [4]:
outliers = []
def detect_outliers_zscore(data):
    thres = 3
    mean = np.mean(data)
    std = np.std(data)
    # print(mean, std)
    for i in data:
        z_score = (i-mean)/std
        if (np.abs(z_score) > thres):
            outliers.append(i)
    return outliers # Driver code

In [5]:
sample_outliers_X1 = detect_outliers_zscore(data['X1'])
sample_outliers_X2 = detect_outliers_zscore(data['X2'])
sample_outliers_X3 = detect_outliers_zscore(data['X3'])
sample_outliers_Y = detect_outliers_zscore(data['Y'])
print("Outliers in X1 column from Z-scores method : ", sample_outliers_X1)
print("Outliers in X2 column from Z-scores method : ", sample_outliers_X2)
print("Outliers in X3 column from Z-scores method : ", sample_outliers_X3)
print("Outliers in Y column from Z-scores method : ", sample_outliers_Y)

Outliers in X1 column from Z-scores method :  []
Outliers in X2 column from Z-scores method :  []
Outliers in X3 column from Z-scores method :  []
Outliers in Y column from Z-scores method :  []


Missing Values Treatement

In [6]:
print(data.isnull().sum())

X1    0
X2    0
X3    0
Y     0
dtype: int64


In [7]:
def normalize(column):
    c_max, c_min = np.max(column), np.min(column)
    for i in range(len(column)):
        column.iloc[i] = (column.iloc[i] - c_min) / (c_max - c_min)
    return c_max, c_min
x1max, x1min = normalize(data['X1'])
x2max, x2min = normalize(data['X2'])
x3max, x3min = normalize(data['X3'])
ymax, ymin = normalize(data['Y'])
data.head()

,X1,X2,X3,Y
0,0.397877,0.293800,0.276454,0.310234
1,0.293800,0.276454,0.310234,0.332801
2,0.276454,0.310234,0.332801,0.414254
3,0.310234,0.332801,0.414254,0.690965
4,0.332801,0.414254,0.690965,0.818465


In [8]:
m1=data['X1'].mean()
m2=data['X2'].mean()
m3=data['X3'].mean()
m4=data['Y'].mean()
data['X1'].fillna(value=m1, inplace=True)
data['X2'].fillna(value=m2, inplace=True)
data['X3'].fillna(value=m3, inplace=True)
data['Y'].fillna(value=m4, inplace=True)
data

,X1,X2,X3,Y
0,0.397877,0.293800,0.276454,0.310234
1,0.293800,0.276454,0.310234,0.332801
2,0.276454,0.310234,0.332801,0.414254
3,0.310234,0.332801,0.414254,0.690965
4,0.332801,0.414254,0.690965,0.818465
...,...,...,...,...
2176,0.354626,0.350803,0.305212,0.228524
2177,0.350803,0.305212,0.228524,0.217797
2178,0.305212,0.228524,0.217797,0.246298
2179,0.228524,0.217797,0.246298,0.324384


In [9]:
from sklearn.model_selection import train_test_split
dataframe = data.values
X, y = dataframe[:, :-1], dataframe[:, -1]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 10)
x_train

array([[0.32324327, 0.58058258, 0.62138027],
       [0.63507461, 0.41339762, 0.35314256],
       [0.64848364, 0.60414824, 0.51333771],
       ...,
       [0.61011098, 0.44563636, 0.4770192 ],
       [0.22641294, 0.17160709, 0.20062195],
       [0.4363071 , 0.41339762, 0.37793501]])

In [10]:
m1=random.uniform(-10,10)
m2=random.uniform(-10,10)
m3=random.uniform(-10,10)
c=random.uniform(-10,10)
eta=0.01
max_iter=1000

In [11]:
def gradient(m1,m2,m3,c,x1,x2,x3,y):
  m1_deriv=-1*(y-m1*x1-m2*x2-m3*x3-c)*x1
  m2_deriv=-1*(y-m1*x1-m2*x2-m3*x3-c)*x2
  m3_deriv=-1*(y-m1*x1-m2*x2-m3*x3-c)*x3
  c_deriv=-1*(y-m1*x1-m2*x2-m3*x3-c)
  return m1_deriv,m2_deriv,m3_deriv,c_deriv

In [12]:
for iter in range(max_iter):
  for sample in range(x_train.shape[0]):
    del_m1,del_m2,del_m3,del_c=gradient(m1,m2,m3,c,x_train[sample,0],x_train[sample,1],x_train[sample,2],y_train[sample])
    m1=m1-eta*del_m1
    m2=m2-eta*del_m2
    m3=m3-eta*del_m3
    c=c-eta*del_c
print('Model parameters m1, m2, m3 and c : ',m1,m2,c)

Model parameters m1, m2, m3 and c :  -0.21748361014645906 0.041550607301544355 0.1538395878338858


Model parameters m1, m2, m3 and c :  -0.21748361014645906 0.041550607301544355 0.1538395878338858

In [13]:
y_pred_train = []
for sample in range(x_train.shape[0]):
  x1=x_train[sample,0]
  x2=x_train[sample,1]
  x3=x_train[sample,2] 
  y=y_train[sample]
  y_p_tr = (m1 * x1) + (m2 * x2) + (m3 * x3) + c
  y_pred_train.append(y_p_tr)
y_pred_train = np.array(y_pred_train)

In [14]:
y_pred_test = []
for sample in range(x_test.shape[0]):
  x1=x_test[sample,0]
  x2=x_test[sample,1]
  x3=x_test[sample,2] 
  y=y_test[sample]
  y_p_te = (m1 * x1) + (m2 * x2) + (m3 * x3) + c
  y_pred_test.append(y_p_te)
y_pred_test = np.array(y_pred_test)

In [15]:
#Training Accuracies
mse = mean_squared_error(y_train, y_pred_train) 
print('Mean Square Error', mse)
rmse = math.sqrt(mean_squared_error(y_train, y_pred_train)) 
print('Root Mean Square Error', rmse)
mae=mean_absolute_error(y_train, y_pred_train)
print('Mean Absolute Error', mae)

Mean Square Error 0.013909085207057836
Root Mean Square Error 0.11793678479192926
Mean Absolute Error 0.08157461836796978


In [16]:
#Testing Accuracies
mse = mean_squared_error(y_test, y_pred_test)
print('Mean Square Error', mse)
rmse = math.sqrt(mean_squared_error(y_test, y_pred_test)) 
print('Root Mean Square Error', mse) 
mae=mean_absolute_error(y_test, y_pred_test)
print('Mean Absolute Error', mae)

Mean Square Error 0.01281352108292449
Root Mean Square Error 0.01281352108292449
Mean Absolute Error 0.08077037730340854


In [18]:
X1=input('Enter Load Value at Time T1: ')
X2=input('Enter Load Value at Time T2: ')
X3=input('Enter Load Value at Time T3: ')
Y=m1*float(X1)+m2*float(X2)+m3*float(X3)+c
print('Power Load at Time T : ',Y)

Enter Load Value at Time T1: 4567
Enter Load Value at Time T2: 5736
Enter Load Value at Time T3: 5443
Power Load at Time T :  3923.5087178851486
